In [1]:
import numpy as np;
import sys;
import os;
import glob;
import random;
import wavio;
from random import shuffle;

## putting speech signal in the head and extending clip to 1.5 Sec

## putting speech signal in the middle and extending clip to 1.5 Sec

## putting speech signal in the end and extending clip to 1.5 Sec

## mix with white noises